# Money class demo

Jeg har brug for en funktionalitet i min kode hvor jeg kan repræsenterer penge.  

### Problem:
Jeg kan gemme beløbet i en `int` eller en `float` 

````
    amount = 20
    amount = 20.95
````

Men jeg kan ikke samtidig gemme hvilken valuta jeg arbejder med (feks. "DKR", "EUR"). 

Jeg kan heller ikke umiddelbart håndterer om jeg skal kunne arbejde med `int` eller `float`, eller håndterer andre issues i forhold til præcision når det drejer sig om floats. Denne rodet kodedel kan jeg **Abstraherer** væk vha. **Encapsulation**

### Datamodel Protokollen
Jeg vil gerne have at mit Money object opfører sig som en int eller en float, og vil altså stadig gerne kunne bruge **+,-,*,/**. Jeg vil også gerne kunne bruge de indbyggede functioner `int()`og `float()` til at konverterer beløbet fra den ene til den anden datatype Derfor bliver jeg nød til at implementerer protokollen for dette i mit object. 


| attribute                                                        | Hvad skal den bruges til    |
| ---------------------------------------------------------------- | -------------------------------------------------------------------------- |
| `amount`                                                         | Den numeriske værdi (feks. 100, eller 2)   |
| `currency`                                                       | ISO-4217 code (`"USD"`, `"EUR"`, …) valuta |                                                                     |


In [ ]:
from numbers import Real   # keeps isinstance checks simple

class Money:
    # ────────────────────────────────────────────
    # Constructor and stored attributes
    # ────────────────────────────────────────────
    def __init__(self, amount, currency="USD"):
        # store as integer “cents” to avoid float-rounding issues
        self.amount = int(amount)
        self.currency = currency

    # ────────────────────────────────────────────
    # Conversion to built-ins
    # ────────────────────────────────────────────
    def __int__(self):
        return self.amount

    def __float__(self):
        return self.amount / 100.0

    # ────────────────────────────────────────────
    # String / repr for debugging and printing
    # ────────────────────────────────────────────
    def __repr__(self):           # Money(1999, 'USD')
        return f"Money({self.amount}, '{self.currency}')"

    def __str__(self):            # $19.99
        symbol = {"USD": "$", "EUR": "€"}.get(self.currency, "")
        return f"{symbol}{self.amount/100:,.2f}"

    # ────────────────────────────────────────────
    # Arithmetic protocol methods
    # ────────────────────────────────────────────
    def _require_same_currency(self, other):
        if self.currency != other.currency:
            raise TypeError(
                f"Currency mismatch: {self.currency} vs {other.currency}"
            )

    def __add__(self, other):
        if isinstance(other, Money):
            self._require_same_currency(other)
            return Money(self.amount + other.amount, self.currency)
        return NotImplemented

    def __sub__(self, other):
        if isinstance(other, Money):
            self._require_same_currency(other)
            return Money(self.amount - other.amount, self.currency)
        return NotImplemented

    def __mul__(self, factor):
        if isinstance(factor, Real):
            return Money(int(self.amount * factor), self.currency)
        return NotImplemented

    # Enable `3 * money`
    __rmul__ = __mul__

    def __truediv__(self, divisor):
        if isinstance(divisor, Real):
            return Money(int(self.amount / divisor), self.currency)
        return NotImplemented

    # Natural equality comparisons
    def __eq__(self, other):
        return (
            isinstance(other, Money)
            and self.amount == other.amount
            and self.currency == other.currency
        )
